In [55]:
import pandas as pd
import geopandas as  gpd

In [56]:
df = pd.read_csv('bronze/Properties.csv', encoding='utf-8', sep=';')
df

,property_url,broker,district,price,type,city,state,ref,category,area,lat,long,num_bedroom,num_bathroom,num_garage,added_date,sold_date
0,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 08,Consulte-nos,Padrão,Maringa,PR,5363.0,Apartamento,"55,34 m²",-23.446941,-51.919694,2,1,-,2025-04-15,2025-08-27
1,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 01,"R$ 90.000,00",Padrão,Maringá,PR,31223.0,Apartamento,"48,40 m²",-23.421000,-51.933056,1,1,-,2025-04-15,NaN
2,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 01,"R$ 126.312,10",Padrão,Maringá,PR,35423.0,Apartamento,"66,36 m²",-23.421000,-51.933056,1,1,-,2025-04-15,NaN
3,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Jardim Barcelona,"R$ 135.000,00",Padrão,Maringá,PR,82763.0,Terreno,"300,00 m²",-23.459239,-51.981950,-,-,-,2025-04-15,NaN
4,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Jardim Novo Paulista,"R$ 140.000,00",Padrão,Maringá,PR,48803.0,Terreno,"200,00 m²",-23.408781,-51.865269,-,-,-,2025-04-15,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,https://www.pedrogranado.com.br/alugar/maringa...,Pedro Granado Imóveis,Jardim São Francisco,"R$ 350.000,00 (V) - R$ 2.100,00 (L)",Casa,Maringá,PR,25553,Casa/Sobrado,"108,32 m²",NaN,NaN,3,3,1,2025-08-27,NaN
2691,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Jardim América,"R$ 600.000,00",Casa,Maringá,PR,93101,Casa/Sobrado,"152,48 m²",NaN,NaN,3,3,1,2025-08-27,NaN
2692,https://www.leloimoveis.com.br/imovel/casa-res...,Lelo Imóveis,Jardim Olimpico,"R$ 550.000,00",NaN,Maringa,PR,79900.13412,Casa,"152,35m²",-23.413055,-51.979418,3,NaN,4,2025-08-27,NaN
2693,https://www.leloimoveis.com.br/imovel/apartame...,Lelo Imóveis,Zona 07,"R$ 619.000,00",NaN,Maringa,PR,79900.13166,Apartamento,"195,00m²",-23.418096,-51.927756,2,NaN,2,2025-08-27,NaN


In [57]:
df['district'] = df['district'].str.title()
df['price'] = pd.to_numeric(df['price'].str.replace('R$ ', '').str.replace('.', '').str.replace(',', '.'), errors='coerce')
df['type'] = df['type'].str.title()
df['city'] = df['city'].str.title().str.replace('Maringa', 'Maringá')
df['state'] = df['state'].str.upper()
df['ref'] =  df['ref'].astype(str).str.strip()
df['category'] = df['category'].str.title()
df['area'] = pd.to_numeric(df['area'].str.replace('m²', '').str.replace('.', '').str.replace(',', '.').str.strip(), errors='coerce')
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['long'] = pd.to_numeric(df['long'], errors='coerce')
df['num_bedroom'] = pd.to_numeric(df['num_bedroom'], errors='coerce')
df['num_bathroom'] = pd.to_numeric(df['num_bathroom'], errors='coerce')
df['num_garage'] = pd.to_numeric(df['num_garage'], errors='coerce')
df['added_date'] = pd.to_datetime(df['added_date'], errors='coerce').dt.date
df['sold_date'] = pd.to_datetime(df['sold_date'], errors='coerce').dt.date

In [58]:
map = {
    'Apartamento': 'Apartamento',
    'Apto': 'Apartamento',
    'Apartamento Kitnet': 'Apartamento',
    'Apartamento Duplex': 'Apartamento',
    'Casa': 'Casa',
    'Casa/Sobrado': 'Casa',
    'Casa Em Condomínio': 'Casa',
    'Sobrado': 'Casa',
    'Sobrado Em Condomínio': 'Casa',
    'Prédio': 'Apartamento',
    'Predio': 'Apartamento',
    'Comercial': 'Comercial',
    'Sala Comercial': 'Comercial',
    'Sala Comercial Em Edifício': 'Comercial',
    'Salão Comercial': 'Comercial',
    'Salão': 'Comercial',
    'Barracão': 'Galpão',
    'Galpão': 'Galpão',
    'Terreno': 'Terreno',
    'Terreno Em Condomínio': 'Terreno',
    'Terreno Comercial': 'Terreno',
    'Chácara': 'Fazenda',
    'Chácara Em Área De Lazer': 'Fazenda',
    'Fazenda': 'Fazenda',
    'Sítio': 'Fazenda',
    'Sítios': 'Fazenda',
    'Rural': 'Fazenda',
    'Área': 'Casa',
    'Negócio': 'Comercial',
    'Imóvel A Beira De Rio E Lago': "Fazenda",
    'Sala': 'Comercial'
}

df['category'] = df['category'].replace(map)

In [59]:
df.loc[~pd.isna(df['sold_date']), 'is_sold'] = True
df.loc[pd.isna(df['sold_date']), 'is_sold'] = False

In [60]:
df['days_on_market'] = (pd.to_datetime(df['sold_date']) - pd.to_datetime(df['added_date'])).dt.days

In [61]:
df['price_per_m2'] = df['price'] / df['area']

In [62]:
df_geo = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['long'], df['lat']),
    crs="EPSG:4326"   # WGS84 (sistema comum de lat/long)
)

# 3. Ler o shapefile da cidade (subdistritos)
gdf_districts = gpd.read_file("maringa_shape\maringa.shx")

# ⚠️ Muito importante: garantir que ambos estejam no mesmo CRS
# Se o shapefile estiver em outro CRS, reprojete:
gdf_districts = gdf_districts.to_crs("EPSG:4326")

# 4. Fazer o spatial join (pontos dentro dos polígonos)
df_districts_adjusted = gpd.sjoin(df_geo, gdf_districts[['NM_SUBDIST', 'geometry']], how="left", predicate="within")

In [63]:
df_districts_adjusted.drop(['index_right', 'geometry'], axis=1, inplace=True)

In [64]:
df_districts_adjusted.rename(columns={'district': 'subdistrict',
                                      'NM_SUBDIST': 'district'}, inplace=True)

In [65]:
df_districts_adjusted

,property_url,broker,subdistrict,price,type,city,state,ref,category,area,...,long,num_bedroom,num_bathroom,num_garage,added_date,sold_date,is_sold,days_on_market,price_per_m2,district
0,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 08,NaN,Padrão,Maringá,PR,5363.0,Apartamento,55.34,...,-51.919694,2.0,1.0,NaN,2025-04-15,2025-08-27,True,134.0,NaN,Zona 28
1,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 01,90000.0,Padrão,Maringá,PR,31223.0,Apartamento,48.40,...,-51.933056,1.0,1.0,NaN,2025-04-15,NaT,False,NaN,1859.504132,Zona 1
2,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Zona 01,126312.1,Padrão,Maringá,PR,35423.0,Apartamento,66.36,...,-51.933056,1.0,1.0,NaN,2025-04-15,NaT,False,NaN,1903.437312,Zona 1
3,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Jardim Barcelona,135000.0,Padrão,Maringá,PR,82763.0,Terreno,300.00,...,-51.981950,NaN,NaN,NaN,2025-04-15,NaT,False,NaN,450.000000,Zona 47
4,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Jardim Novo Paulista,140000.0,Padrão,Maringá,PR,48803.0,Terreno,200.00,...,-51.865269,NaN,NaN,NaN,2025-04-15,NaT,False,NaN,700.000000,Zona 36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,https://www.pedrogranado.com.br/alugar/maringa...,Pedro Granado Imóveis,Jardim São Francisco,NaN,Casa,Maringá,PR,25553,Casa,108.32,...,NaN,3.0,3.0,1.0,2025-08-27,NaT,False,NaN,NaN,NaN
2691,https://www.pedrogranado.com.br/comprar/maring...,Pedro Granado Imóveis,Jardim América,600000.0,Casa,Maringá,PR,93101,Casa,152.48,...,NaN,3.0,3.0,1.0,2025-08-27,NaT,False,NaN,3934.942288,NaN
2692,https://www.leloimoveis.com.br/imovel/casa-res...,Lelo Imóveis,Jardim Olimpico,550000.0,NaN,Maringá,PR,79900.13412,Casa,152.35,...,-51.979418,3.0,NaN,4.0,2025-08-27,NaT,False,NaN,3610.108303,Zona 19
2693,https://www.leloimoveis.com.br/imovel/apartame...,Lelo Imóveis,Zona 07,619000.0,NaN,Maringá,PR,79900.13166,Apartamento,195.00,...,-51.927756,2.0,NaN,2.0,2025-08-27,NaT,False,NaN,3174.358974,Zona 9


In [66]:
df_districts_adjusted.to_csv('silver/Properties_silver.csv', index=False, sep=';')